<a href="https://colab.research.google.com/github/haziqzikry/business-intelligence-analytics-project/blob/main/0_amazon_reviews_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 22.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [3]:
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Define the categories
categories = [
    # "Digital_Music", - Done
    # "Magazine_Subscriptions", - Done
    # "Movies_and_TV",
    "Musical_Instruments",
    "Sports_and_Outdoors",
    "Toys_and_Games",
    "Video_Games"
]

In [6]:
# Function to load, clean, and transform the dataset
def load_and_clean_category(category):
    # Load the dataset
    dataset = load_dataset(
        "McAuley-Lab/Amazon-Reviews-2023",
        f"raw_review_{category}",
        trust_remote_code=True,
    )

    # Determine available splits
    available_splits = list(dataset.keys())
    print(f"Available splits for {category}: {available_splits}")

    # Use the first available split (assuming it has data)
    split_name = available_splits[0]

    # Convert to pandas DataFrame
    df = pd.DataFrame(dataset[split_name])

    # Convert 'timestamp' column to datetime and filter by 2022-2023
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df = df[(df['timestamp'] >= '2022-01-01') & (df['timestamp'] <= '2023-12-31')]
        # df['timestamp'] = df['timestamp'].dt.date  # Show only the date part

    # Remove the 'images' column
    if 'images' in df.columns:
        df = df.drop(columns=['images'])

    # Rename columns
    df = df.rename(columns={
        'title': 'review_title',
        'text': 'review_text',
        'timestamp': 'date'
    })

    # Drop missing values
    df = df.dropna()

    # Add 'Category' column
    df['Category'] = category.replace('_', ' ')

    return df

In [ ]:
# Load and clean datasets for all categories with a progress bar
category_dataframes = {}
for category in tqdm(categories, desc="Loading and cleaning datasets"):
    category_dataframes[category] = load_and_clean_category(category)

Loading and cleaning datasets:   0%|          | 0/5 [00:00<?, ?it/s]

Available splits for Movies_and_TV: ['full']


In [ ]:
# # Export the cleaned dataframes to CSV files
# for category, df in category_dataframes.items():
#     df.to_csv(f"{category}_reviews_processed.csv", index=False)

In [ ]:
# Export the cleaned dataframes to CSV files in Google Drive
drive_path = '/content/drive/MyDrive/Amazon_Reviews_2023/Reviews/'
for category, df in category_dataframes.items():
    df.to_csv(drive_path + f"{category}_reviews_processed.csv", index=False)

In [ ]:
# # Example: Access the cleaned dataframe for Digital_Music
# df_test = category_dataframes['Digital_Music']
# df_test


In [ ]:
#exported csv will have null value even after export